In [1]:
import numpy as np
from loadData import DataLoader
import torch
import syft as sy
import copy
from torch import nn, optim

In [2]:
"""""
Attributes:
    -denseOutputData
    -sparseOutputData
    -denseInputData
    -denseSampleIndex
    -sparseInputData
    -sparseSampleIndex
    -allSamples
    -folds
"""""
data = DataLoader(datasetName='static')

In [3]:
data.denseInputData.dtype

dtype('float64')

In [4]:
#sparse-t kéne hasztnálni, ez sok memória
data.sparseOutputData = data.sparseOutputData.tocoo().astype('float64') 

In [5]:
hook = sy.TorchHook(torch)

In [6]:
nworkers = 10
workers = []
for i in range(nworkers):
    workers.append(sy.VirtualWorker(hook, id=str(i)))

In [7]:
data.sparseOutputData.shape

(456331, 1310)

In [8]:
x = torch.tensor(data.denseInputData, requires_grad=True)
i = torch.LongTensor([data.sparseOutputData.row,
                          data.sparseOutputData.col])
v = torch.FloatTensor(data.sparseOutputData.data)
y = torch.sparse.FloatTensor(i, v, torch.Size(data.sparseOutputData.shape)).requires_grad_(True)
in_dim  = x.size()[0]
out_dim = y.size()[0]
latent_dim = 500

In [9]:
print(data.sparseOutputData.shape)
print(y.size())

(456331, 1310)
torch.Size([456331, 1310])


In [63]:
def splice_sparse(sparseMatrix, istart, iend):
    cond = np.logical_and(sparseMatrix._indices()[0]>=istart, sparseMatrix._indices()[0]<iend)
    rows = np.select(cond, sparseMatrix._indices()[0])
    cols = np.select(cond, sparseMatrix._indices()[1])
    values = np.select(cond, sparseMatrix._values())
    print(rows.shape)
    print(cols.shape)
    print(values.shape)

In [ ]:
splice_sparse(y, 3 * data_per_worker, (3+1) * data_per_worker)

In [15]:
x_pointers = []
y_pointers = []
#data_per_worker = int(x.size()[0] / nworkers)

data_per_worker = 100
for i in range(1):
    print("from:",(i) * data_per_worker, "to", (i+1) * data_per_worker)
    x_pointers.append(x[i * data_per_worker: (i+1) * data_per_worker].send(workers[i]))
    y_pointers.append(splice_sparse(y, i * data_per_worker, (i+1) * data_per_worker))

from: 0 to 100
tensor([[  345,   424,  1730,  ..., 27808, 29156, 30262],
        [    0,     0,     0,  ...,  1309,  1309,  1309]])


In [ ]:
class TrunkNet(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(in_dim, out_dim)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)